In [1]:
# Notebook to extract timeseries from 

In [ ]:
import sys

import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mpc

import cartopy.feature as ccft
import cartopy.crs as ccrs

import pandas as pd
import pykrige as pk
import pykrige.kriging_tools as kt

import gstools as gs

In [2]:
init_dir = os.getcwd()
data_dir = f'{init_dir}/data'

In [3]:
z_dat = pd.read_excel(f'{data_dir}/obs_ts_1940.xlsx')
loc_dat = pd.read_excel(f'{data_dir}/tr_meta.xlsx')
true_cols = z_dat.columns[1:]
z_dat# Set a custom index
loc_dat.set_index('Series Name', inplace=True)
z_dat.set_index('Year', inplace=True)
true_cols = true_cols[1:]

In [4]:
init_dir = os.getcwd()

dat_str = f'{data_dir}/gCNN_1kE_ret.nc'

dat = xr.open_dataset(dat_str)

In [5]:
dat = dat.rename(
    {'__xarray_dataarray_variable__':'d18op'}
        )

In [6]:
def f(name, year, data = dat):
    
    """
    Function to return the time series of a study site using the PRYSM model. 
    See Dee et al., (2015); Evans (2007).
    
    Inputs:
        n: name of study site
        data: xr datarray to be used to calculate and extract values
    
    Outputs:
        site_d18: Timeseries of predicted d18Otr 
    
    """
    
    lat0 = loc_dat.loc[name]['Lat']
    lon0 = loc_dat.loc[name]['Lon']


    dat_ = data.d18op.sel(latitude = lat0, method='nearest')
    dat_ = dat_.sel(longitude = lon0, method='nearest')
    dat_ = dat_.sel(year=year)

    val = round(float(dat_), 3)
    
    return val

In [8]:
f('ball_v', 2000)

30.362

In [10]:
# Stack the DataFrame and remove rows with all NaN values
z_stack = z_dat.stack().dropna()

In [11]:
data_list = []

for i, meta in enumerate(z_stack.index):
    
    year = meta[0]
    name = meta[1]
    
    z = z_stack.loc[meta]

    lat = loc_dat.loc[name].Lat
    lon = loc_dat.loc[name].Lon
    
    o = f(name, year, data=dat)
    
        # Append the data to the list
    data_list.append({
        "year": year,
        "name": name,
        "gCNN_d18o": o
    })

# Create a DataFrame from the list
result_df = pd.DataFrame(data_list)

In [18]:
result_df[result_df['name'] == 'jenkins']

,year,name,gCNN_d18o
2,1940,jenkins,23.038
8,1941,jenkins,23.038
14,1942,jenkins,23.038
20,1943,jenkins,23.038
26,1944,jenkins,23.038
...,...,...,...
589,2002,jenkins,23.038
599,2003,jenkins,23.038
609,2004,jenkins,23.038
619,2005,jenkins,23.038


In [35]:
result_df.to_csv(f'{data_dir}/gCNN_ts.csv')